In [1]:
import io
import re
import os


import string
#import tempfile 


from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
from PyPDF2 import PdfFileReader, PdfFileWriter
from pdfminer.layout import LAParams

from pdfminer.high_level import extract_text_to_fp

from arabic_reshaper import ArabicReshaper
from bidi.algorithm import get_display

In [4]:
configuration = {
    'delete_harakat': True,
    'delete_tatweel ': True,
}

year_folder_id_dict = {'2010':'113DAB_g6P3b-gbrpFog682EfdILcBMt6','2011':'1Yh9610KUMxZLNDiyUOnng-7PbhlYbt3x',
                       '2012':'1TEWAyyxf6dJ3K6hMiliayJ4VV4ilvwIH','2013':'1KWNg6Me11spV-JRFOzwlvNpvoYMLI59C',
                       '2014':'1IgDuaPlc9b-MdP6VELFXcTffqSRHYPU5','2015':'1CrHueAEH76p95tIOryMrdu5nV-qW702M',
                       '2016':'1hpViY56I20A6k6EeQGrOn6QG11j7gb58','2017':'1H2HXwALMyzxBIMJyluYkTHJKplqGIpUF',
                       '2018':'1-jlTY-u_izedKKNenevC3kgM6yaKR7b7','2019':'1Qm-Sse4vbOx2IOAILUCTvXwqShWlynQn'}

events_folder_id_dict = {'2010':'1CPmsZoNqsDslYUDuMMYAiTRdgcbjuIj6','2011':'1FQenIULvAsgQN4sLSpcLdjjfStfeIvga',
                       '2012':'1ABi7lAAmlARbrGlBJv8Tz6IFXVSYzLdR','2013':'1MMg8bUfZj3eZf_gAlr3k2N9_LWmO5DWr',
                       '2014':'135rCpr_1_N00nmsKOcyWjmQPmT_n8I96','2015':'1Gjm0qtTXL-21qNQJtloxNGtJq3Q1Xlyq',
                       '2016':'1qRUeOe4mFx5ofuYHxiqL4u1HvqxLCz96','2017':'15DY5sNvP8O2jgus7Qythy8Nzeeo7cZAI',
                       '2018':'1AKHSz-t3vkUsLE2IC4TdrMUznjNC3YpU','2019':'1qOX3DQFtU_j4czyUMndjpn9c0crZ8ue1'}
events_folder_id = '165-gELn98fyKGaWnmWSkJuUuz1aj8XMg'

In [5]:
def convert_text(word):
    regex = r"({.*?})"
    word = re.findall(regex,str(word.encode('WINDOWS-1251',errors='namereplace')))
    cleaned = []
    for char in word:
        char = char.replace('{','').replace('}','')
        cleaned.append(char)
    
    cleaned_word = []
    for x in cleaned:
       if len(x.split()) > 2:
        cleaned_word.append(' '.join(i for i in x.split()[:3]))
        
    return cleaned_word

In [6]:
section_keyword = ['لبنان','اقتصاد','مجتمع','سياسة']
section_keyword_encoded = [convert_text(x) for x in section_keyword]

In [7]:
with open('keywords.txt','r') as f:
    txt = f.read().replace('\n',' ')
    Keywords = [x for x in txt.split() if len(x) > 1 ]
    
Keywords_encoded = [convert_text(x) for x in Keywords]

In [8]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=creds)

In [9]:
corrupted_files = []

for year in year_folder_id_dict.keys():
    folder_id = year_folder_id_dict[year]
    results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
    files = results.get('files', [])
    # Iterate through the files
    for file in files:
        file_id = file.get('id')
        file_name = file.get('name')
        
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            #print("Download %d%%." % int(status.progress() * 100))
        pdf_content = fh.getvalue()
        with open('new.pdf', 'wb') as f:
            f.write(pdf_content)
        try:
            pdf = PdfFileReader(open('new.pdf', 'rb'))
            print('oki papa')
        except:
            corrupted_files.append(file_name)
            try:
                service.files().delete(fileId=file['id']).execute()
            except HttpError as error:
                print('An error occurred: %s' % error)

An error occurred: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/17VCtujDMFjDNW-iK3LdWyTb44gYDZoMc? returned "The user does not have sufficient permissions for this file.". Details: "[{'message': 'The user does not have sufficient permissions for this file.', 'domain': 'global', 'reason': 'insufficientFilePermissions'}]">


In [3]:
corrupted_files

NameError: name 'corrupted_files' is not defined

In [10]:
for year in year_folder_id_dict.keys():
    folder_id = year_folder_id_dict[year]
    results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
    files = results.get('files', [])
    
    events_folder = service.files().list(q=f"'{events_folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
    events_files = events_folder.get('files', [])
    # Iterate through the files
    for file in files:
        if file in events_files:
            print(F"file with name {file.get('name')} already exists in the folder")
        else:
            file_id = file.get('id')
            file_name = file.get('name')
            
            request = service.files().get_media(fileId=file_id)
            fh = io.BytesIO()
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                #print("Download %d%%." % int(status.progress() * 100))
            pdf_content = fh.getvalue()
            with open('new.pdf', 'wb') as f:
                f.write(pdf_content)
            try :
                pdf = PdfFileReader(open('new.pdf', 'rb'))

                for page in range(1,pdf._get_num_pages()):
                    with open('new.pdf', 'rb') as f:
                        laparams = LAParams(char_margin=1.0, line_margin=0.5, word_margin=0.1)
                        outfp = io.StringIO()
                        outfp.seek(0)
                        extract_text_to_fp(f, outfp, laparams=laparams, page_numbers={page})
                        outfp.seek(0)
                        text = outfp.read()
                        reshaper = ArabicReshaper(configuration=configuration)
                        corrected_text = get_display(text)
                        reshaped_text = reshaper.reshape(corrected_text)
                        
                        
                        section = ''.join(x for x in reshaped_text[:60] if x not in string.digits).strip().replace('\n',' ').replace('ـ', '')
                        section = [x for x in section.split() if len(x) > 2]
                        section_encoded = [convert_text(x) for x in section]
                        
                        if any(x in section_encoded for x in section_keyword_encoded):

                                reshaped_text = reshaped_text.strip().replace('\n',' ').replace('ـ', '')
                                encoded_text = []
                                for x in reshaped_text.split():
                                    if len(x) > 1:
                                        encoded_text.append(convert_text(x))
                                
                                if any(x in encoded_text for x in Keywords_encoded):
                                    name = file_name.replace('.pdf', f'-page-{int(page)+1}.pdf')
                                    with open(name,'wb') as new_page:
                                        pdf_writer = PdfFileWriter()
                                        pdf_writer.addPage(pdf.getPage(page))
                                        pdf_writer.write(new_page)
                                    
                                    file_metadata = {'name': name,'parents':['165-gELn98fyKGaWnmWSkJuUuz1aj8XMg']}
                                    media = MediaFileUpload(name, mimetype='application/pdf')
                                    service.files().create(body=file_metadata,
                                                                        media_body=media,
                                                                        fields='id').execute()    
                                    os.remove(path=name)
                                    print('oki papa', name)
            except:
                corrupted_files.append(file_name)
                print('Error', file_name)

Error 2010-8-20.pdf
Error 2010-8-19.pdf
Error 2010-8-18.pdf
Error 2010-8-17.pdf
Error 2010-8-14.pdf
Error 2010-8-13.pdf
Error 2010-8-12.pdf
Error 2010-8-11.pdf
Error 2010-8-9.pdf
Error 2010-8-7.pdf
Error 2010-8-6.pdf
Error 2010-8-5.pdf
Error 2010-8-4.pdf
Error 2010-8-3.pdf
Error 2010-8-2.pdf
Error 2010-9-30.pdf
Error 2010-9-29.pdf
Error 2010-9-28.pdf
Error 2010-9-27.pdf
Error 2010-9-25.pdf
Error 2010-9-24.pdf
Error 2010-9-23.pdf
Error 2010-9-22.pdf
Error 2010-9-21.pdf
Error 2010-9-20.pdf
Error 2010-9-18.pdf
Error 2010-9-17.pdf
Error 2010-9-16.pdf
Error 2010-9-15.pdf
Error 2010-9-14.pdf
Error 2010-9-13.pdf
Error 2010-9-10.pdf
Error 2010-9-9.pdf
Error 2010-9-8.pdf
Error 2010-9-7.pdf
Error 2010-9-6.pdf
Error 2010-9-4.pdf
Error 2010-9-3.pdf
Error 2010-9-2.pdf
Error 2010-9-1.pdf
Error 2010-10-30.pdf
Error 2010-10-29.pdf
Error 2010-10-28.pdf
Error 2010-10-27.pdf
Error 2010-10-26.pdf
Error 2010-10-25.pdf
Error 2010-10-23.pdf
Error 2010-10-22.pdf
Error 2010-10-21.pdf
Error 2010-10-20.pdf
Error

KeyboardInterrupt: 